In [ ]:
from scipy import stats
from multiprocessing import Pool
from statsmodels.stats.multitest import fdrcorrection

# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/mouse_ASm6A/count/")
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/")
##########
def preprocess_df():
#     ip_file, input_file = "D395A_IP1.readcounts.txt", "D395A_Input1.readcounts.txt"
#     ip_file, input_file = "WT_IP1.readcounts.txt", "WT_Input1.readcounts.txt"
    # ip_file, input_file = "Nc1-ip_L4_711D01.readcounts.txt", "Nc1-inp_L4_709D01.readcounts.txt"
    # ip_file, input_file = "Fus2-ip_L4_712D01.readcounts.txt", "Fus2-inp_L4_710D01.readcounts.txt"
    # ip_file, input_file = "NC-IP_L1_706D07.readcounts.txt", "NC-inp_L1_704D07.readcounts.txt"
    ip_file, input_file = "Fus-IP_L1_707D07.readcounts.txt", "Fus-inp_L1_705D07.readcounts.txt"
    df_ip, df_input = pd.read_table(ip_file), pd.read_table(input_file)
    df_results = []
####### METTL3 ####
#     totalNum_list = [76552038, 116235090]
#     totalNum_list = [120990009, 90418006]
####### FUS ####
    totalNum_list = [117018661, 59254724] # KD 
#     totalNum_list = [90201519, 46888237] # NC
#     totalNum_list = [85184165, 78164647] # NC
#    totalNum_list = [84298167, 68023740] # KD
    treat_list, df_list = ['ip', 'input'], [df_ip, df_input]
    for i in range(len(df_list)):
        df, treat, total_num = df_list[i], treat_list[i], totalNum_list[i]
        df = df[['contig', 'position', 'refAllele', 'altAllele', 'refCount', 'altCount']]
#         df = df[(df['refCount']>=2) & (df['altCount']>=2)]
#         df = df[(df['refCount'] + df['altCount'])>=10]
        df['refCount_%s' % treat] = df['refCount']
        df['altCount_%s' % treat] = df['altCount']
        df['refRPKM_%s' % treat] = (df['refCount'] / total_num) * 1000000000
        df['altRPKM_%s' % treat] = (df['altCount'] / total_num) * 1000000000
        df_results.append(df)
    return df_results[0], df_results[1]

### 
result_dir = "Hypothesis/Fisher/"
os.system("mkdir -p %s" % result_dir)
result_file = os.path.join(result_dir, "KD_2.txt") ############
###
df_ip, df_input = preprocess_df()
print(len(df_ip), len(df_input))
df = df_ip.merge(df_input, on=['contig', 'position', 'refAllele', 'altAllele'], how='left').dropna(how="any")
print(len(df))
df['refRPKM_ratio'] = df['refRPKM_ip'] / df['refRPKM_input']
df['altRPKM_ratio'] = df['altRPKM_ip'] / df['altRPKM_input']
## ref/(ref+alt)
df['allelicRatio'] = df['refRPKM_ratio'] / (df['altRPKM_ratio'] + df['refRPKM_ratio'])
pvalue_list , odds_list = [], []
for i,j in df.iterrows():
    a,b = round(j['refRPKM_ip'])+0.5, round(j['altRPKM_ip'])+0.5
    c,d = round(j['refRPKM_input'])+0.5, round(j['altRPKM_input'])+0.5
    oddsratio,pvalue=stats.fisher_exact([[a,b], [c,d]])
    pvalue_list.append(pvalue)
    odds_list.append(oddsratio)
qvalue_list = list(fdrcorrection(pvalue_list)[1])
df['pvalue'],df['oddsratio'],df['qvalue'] = pvalue_list,odds_list,qvalue_list
df_sub=df[['contig','position','refAllele','altAllele','refRPKM_ratio','altRPKM_ratio', 'allelicRatio', 'pvalue', 'oddsratio','qvalue','refCount_ip','altCount_ip','refCount_input','altCount_input']]
df_sub.to_csv(result_file, sep="\t", index=False)


#### 1. select significant ASm6As
#### 2. only select heterozygous sites which satisfy the lowest read number

# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/mouse_ASm6A/count/Hypothesis/Fisher/")
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/Fisher/")
txt_list = glob.glob("*.txt")

result_dir = "sig/"
# result_dir = "filter_byCount/"
# result_dir = "heter_sites/"
os.system("mkdir -p %s" % result_dir)
########

for txt in txt_list:
    df = pd.read_table(txt)
    ### input >= 2;10
    df = df[(df['refCount_input']>=2) & (df['altCount_input']>=2)]
    df = df[(df['refCount_input'] + df['altCount_input'])>=10]
    ### ip >= 10;
    df = df[(df['refCount_ip'] + df['altCount_ip'])>=10]
    ### qvalue < 0.5
    df = df[df['qvalue'] < 0.05]
    result_file = os.path.join(result_dir, txt)
    df.to_csv(result_file, sep="\t", index=False)